## 1) Reliable Users


<img src='http://www.indiaeducation.net/imagesvr_ce/698/Critics%201.jpg' />
<p style="text-indent: 40px">For the first part of this assignment you should find out the top 1000 reliable users. In order to do that you would need tags' relevance score from genome-scores.csv file. Assuming the relevance scores of movies' tags in genome-scores.csv are correct, we can find out if a user's tag to a particular movie is relevant or not. If we get the total of those relevance scores for each user then we can rank the them according to this criteria.</p>

<p style="text-indent: 40px">One caveat is that relevance scores are between 0 and 1. We want to scale it between -1 and 1 and also we want to punish more if the issued tag is not relevant. First multiply relevance with 20 and then substract 12 so relevance score range will be between -12 and 8 and then apply hyperbolic tangent. That way relevance's range will be between -1 and 1.</p>

<p style="text-indent: 40px"> Note: You should use PySpark's <b>tanh</b> function. Also use caching appropriately in order to be able to get the results in a reasonable time.</p>
<p style="text-indent: 40px"> Caution: Do not use RDDs. Every operation should be done on the executors not in the driver machine.</p>
<p style="text-indent: 40px">Important: Do not discuss the solution with your friends. <b>Plagiarism</b> will not be tolerated and issue will be referred to the <b>disciplinary committee</b>.</p>

<div>
    <img src="http://image.prntscr.com/image/1f91352835964c31b04be03da7d53581.jpg" width=500>

    <center><strong>Figure 1: Tanh function.</strong></center>
</div>

# Data
#### https://www.kaggle.com/grouplens/movielens-20m-dataset

In [18]:
import ibmos2spark

### CREDENTIALS

configuration_name = 'os_b0d31744b6c54747aec215514fff27a4_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [19]:
tags = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('tag.csv', 'cs340spring20184b12f92a8d204278af3959660617b691'))
tags.show()

+------+-------+-----------------+-------------------+
|userId|movieId|              tag|          timestamp|
+------+-------+-----------------+-------------------+
|    18|   4141|      Mark Waters|2009-04-24 18:19:40|
|    65|    208|        dark hero|2013-05-10 01:41:18|
|    65|    353|        dark hero|2013-05-10 01:41:19|
|    65|    521|    noir thriller|2013-05-10 01:39:43|
|    65|    592|        dark hero|2013-05-10 01:41:18|
|    65|    668|        bollywood|2013-05-10 01:37:56|
|    65|    898| screwball comedy|2013-05-10 01:42:40|
|    65|   1248|    noir thriller|2013-05-10 01:39:43|
|    65|   1391|             mars|2013-05-10 01:40:55|
|    65|   1617|         neo-noir|2013-05-10 01:43:37|
|    65|   1694|            jesus|2013-05-10 01:38:45|
|    65|   1783|    noir thriller|2013-05-10 01:39:43|
|    65|   2022|            jesus|2013-05-10 01:38:45|
|    65|   2193|           dragon|2013-05-10 02:01:54|
|    65|   2353|conspiracy theory|2013-05-10 02:01:06|
|    65|  

In [20]:
genome_tags = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('genome_tags.csv', 'cs340spring20184b12f92a8d204278af3959660617b691'))
genome_tags.show()

+-----+---------------+
|tagId|            tag|
+-----+---------------+
|    1|            007|
|    2|   007 (series)|
|    3|   18th century|
|    4|          1920s|
|    5|          1930s|
|    6|          1950s|
|    7|          1960s|
|    8|          1970s|
|    9|          1980s|
|   10|   19th century|
|   11|             3d|
|   12|           70mm|
|   13|            80s|
|   14|           9/11|
|   15|        aardman|
|   16|aardman studios|
|   17|       abortion|
|   18|         absurd|
|   19|         action|
|   20|  action packed|
+-----+---------------+
only showing top 20 rows



In [21]:
genome_scores = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('genome_scores.csv', 'cs340spring20184b12f92a8d204278af3959660617b691'))
genome_scores.show()

+-------+-----+-------------------+
|movieId|tagId|          relevance|
+-------+-----+-------------------+
|      1|    1|              0.025|
|      1|    2|              0.025|
|      1|    3|            0.05775|
|      1|    4|            0.09675|
|      1|    5|            0.14675|
|      1|    6|              0.217|
|      1|    7|              0.067|
|      1|    8|            0.26275|
|      1|    9|              0.262|
|      1|   10|              0.032|
|      1|   11|              0.577|
|      1|   12|            0.11625|
|      1|   13|              0.188|
|      1|   14|0.00800000000000001|
|      1|   15|            0.03675|
|      1|   16|            0.28175|
|      1|   17|0.00700000000000001|
|      1|   18|             0.1105|
|      1|   19|             0.6705|
|      1|   20|             0.1845|
+-------+-----+-------------------+
only showing top 20 rows



In [22]:
ratings = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('rating.csv', 'cs340spring20184b12f92a8d204278af3959660617b691'))
ratings.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
|     1|    112|   3.5|2004-09-10 03:09:00|
|     1|    151|     4|2004-09-10 03:08:54|
|     1|    223|     4|2005-04-02 23:46:13|
|     1|    253|     4|2005-04-02 23:35:40|
|     1|    260|     4|2005-04-02 23:33:46|
|     1|    293|     4|2005-04-02 23:31:43|
|     1|    296|     4|2005-04-02 23:32:47|
|     1|    318|     4|2005-04-02 23:33:18|
|     1|    337|   3.5|2004-09-10 03:08:29|
|     1|    367|   3.5|2005-04-02 23:53:00|
|     1|    541|     4|2005-04-02 23:30:03|
|     1|    589|   3.5|2005-04-02 23:45:57|
|     1|    593|   3.5|2005-04-02 23:31:01|
|     1|    653|     3|2004-09-10 03:08:11|
|     1|    919|   3.5|2004-09-1

In [23]:
movies = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('movie.csv', 'cs340spring20184b12f92a8d204278af3959660617b691'))
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

## Which "User" gave which "Tag" to which "Movie"?

In [24]:
user_tag_movie_joined = tags.join(genome_tags, on="tag")
user_tag_movie_joined.show()

+-----------------+------+-------+-------------------+-----+
|              tag|userId|movieId|          timestamp|tagId|
+-----------------+------+-------+-------------------+-----+
|        dark hero|    65|    208|2013-05-10 01:41:18|  288|
|        dark hero|    65|    353|2013-05-10 01:41:19|  288|
|    noir thriller|    65|    521|2013-05-10 01:39:43|  712|
|        dark hero|    65|    592|2013-05-10 01:41:18|  288|
|        bollywood|    65|    668|2013-05-10 01:37:56|  149|
| screwball comedy|    65|    898|2013-05-10 01:42:40|  894|
|    noir thriller|    65|   1248|2013-05-10 01:39:43|  712|
|             mars|    65|   1391|2013-05-10 01:40:55|  630|
|         neo-noir|    65|   1617|2013-05-10 01:43:37|  700|
|            jesus|    65|   1694|2013-05-10 01:38:45|  579|
|    noir thriller|    65|   1783|2013-05-10 01:39:43|  712|
|            jesus|    65|   2022|2013-05-10 01:38:45|  579|
|           dragon|    65|   2193|2013-05-10 02:01:54|  321|
|conspiracy theory|    6

In [25]:
user_tag_movie = user_tag_movie_joined.select(["userId","tagId","movieId"])
user_tag_movie.show()

+------+-----+-------+
|userId|tagId|movieId|
+------+-----+-------+
|    65|  288|    208|
|    65|  288|    353|
|    65|  712|    521|
|    65|  288|    592|
|    65|  149|    668|
|    65|  894|    898|
|    65|  712|   1248|
|    65|  630|   1391|
|    65|  700|   1617|
|    65|  579|   1694|
|    65|  712|   1783|
|    65|  579|   2022|
|    65|  321|   2193|
|    65|  252|   2353|
|    65|  630|   2662|
|    65|  712|   2726|
|    65|  579|   2840|
|    65|  579|   3052|
|    65|  149|   5135|
|    65| 1043|   6539|
+------+-----+-------+
only showing top 20 rows



## Let's ask every "User": How relevant is that "tag" for that "movie", ha?
<img src='https://heidelblog.net/wp-content/uploads/2013/06/critic.jpg' />

In [26]:
user_tag_movie_relevance = user_tag_movie.join(genome_scores, ((user_tag_movie.tagId==genome_scores.tagId) and (user_tag_movie.movieId == genome_scores.movieId)))
user_tag_movie_relevance.show()

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

# I may look like Python, but ...
# I am actually java

In [27]:
user_tag_movie_relevance = user_tag_movie.join(genome_scores, ((user_tag_movie.tagId==genome_scores.tagId) & (user_tag_movie.movieId == genome_scores.movieId)))
user_tag_movie_relevance.show()

+------+-----+-------+-------+-----+---------+
|userId|tagId|movieId|movieId|tagId|relevance|
+------+-----+-------+-------+-----+---------+
| 11074|    1|   2989|   2989|    1|  0.99975|
| 56174|    1|   2989|   2989|    1|  0.99975|
| 60710|    1|   2989|   2989|    1|  0.99975|
| 86768|    1|   2989|   2989|    1|  0.99975|
|105757|    1|   2989|   2989|    1|  0.99975|
| 88738|   10|  27271|  27271|   10|   0.6955|
|130827|   10|  27271|  27271|   10|   0.6955|
| 59025| 1003|   2124|   2124| 1003|   0.3385|
| 16724| 1007|  91094|  91094| 1007|  0.87125|
| 88738| 1007|  91094|  91094| 1007|  0.87125|
|  1587| 1013|   4878|   4878| 1013|  0.83225|
|  4660| 1013|   4878|   4878| 1013|  0.83225|
| 25737| 1013|   4878|   4878| 1013|  0.83225|
| 27898| 1013|   4878|   4878| 1013|  0.83225|
| 32541| 1013|   4878|   4878| 1013|  0.83225|
| 57763| 1013|   4878|   4878| 1013|  0.83225|
| 61843| 1013|   4878|   4878| 1013|  0.83225|
| 83982| 1013|   4878|   4878| 1013|  0.83225|
| 97195| 1013

# Or...

In [28]:
user_tag_movie_relevance = user_tag_movie.join(genome_scores, on=["tagId","movieId"])
user_tag_movie_relevance.show()

+-----+-------+------+---------+
|tagId|movieId|userId|relevance|
+-----+-------+------+---------+
|    1|   2989| 11074|  0.99975|
|    1|   2989| 56174|  0.99975|
|    1|   2989| 60710|  0.99975|
|    1|   2989| 86768|  0.99975|
|    1|   2989|105757|  0.99975|
|   10|  27271| 88738|   0.6955|
|   10|  27271|130827|   0.6955|
| 1003|   2124| 59025|   0.3385|
| 1007|  91094| 16724|  0.87125|
| 1007|  91094| 88738|  0.87125|
| 1013|   4878|  1587|  0.83225|
| 1013|   4878|  4660|  0.83225|
| 1013|   4878| 25737|  0.83225|
| 1013|   4878| 27898|  0.83225|
| 1013|   4878| 32541|  0.83225|
| 1013|   4878| 57763|  0.83225|
| 1013|   4878| 61843|  0.83225|
| 1013|   4878| 83982|  0.83225|
| 1013|   4878| 97195|  0.83225|
| 1013|   4878|107711|  0.83225|
+-----+-------+------+---------+
only showing top 20 rows



# Remember this ...?
### First multiply relevance with 20 and then substract 12 so relevance score range will be between -12 and 8 and then apply hyperbolic tangent. That way relevance's range will be between -1 and 1.

In [29]:
from pyspark.sql.functions import col, tanh

In [32]:
users_reliability_flat = user_tag_movie_relevance.withColumn("reliability", tanh(user_tag_movie_relevance.relevance*20-12))\
                                            .select(["userId","reliability"])
users_reliability_flat.show()

+------+-------------------+
|userId|        reliability|
+------+-------------------+
| 11074| 0.9999997726676818|
| 56174| 0.9999997726676818|
| 60710| 0.9999997726676818|
| 86768| 0.9999997726676818|
|105757| 0.9999997726676818|
| 88738|  0.957085420564793|
|130827|  0.957085420564793|
| 59025|-0.9999426811764384|
| 16724| 0.9999611915372831|
| 88738| 0.9999611915372831|
|  1587| 0.9998153309323468|
|  4660| 0.9998153309323468|
| 25737| 0.9998153309323468|
| 27898| 0.9998153309323468|
| 32541| 0.9998153309323468|
| 57763| 0.9998153309323468|
| 61843| 0.9998153309323468|
| 83982| 0.9998153309323468|
| 97195| 0.9998153309323468|
|107711| 0.9998153309323468|
+------+-------------------+
only showing top 20 rows



In [33]:
users_reliability_flat.count()

206153

In [34]:
users_reliability = users_reliability_flat.groupBy("userId").avg()
users_reliability.show()

+------+--------------------+
|userId|    avg(reliability)|
+------+--------------------+
|111982|  0.5690469165024034|
| 80258|  0.9567662048431939|
| 49307|  0.9999462608511014|
| 92072|  0.7472656429959093|
| 79849|  0.9999971730256131|
| 89274|0.030434008661444568|
| 11563|  0.6247823679141723|
|119625|  0.7275098901796045|
|  6731|  0.9999996505317048|
| 70170|  0.6410873409222123|
|112480|  0.9999814358015412|
| 59370|  0.8275657303574266|
| 54798|  0.5630558543678916|
| 23918|  0.9999066771110212|
| 56525|  0.9979402195807233|
| 60733|  0.6665034028246085|
|128566|  0.9999995774056201|
| 39581|  0.3594432427158843|
| 79757|  0.9999987745652463|
|129005| 0.24929532350910683|
+------+--------------------+
only showing top 20 rows



In [35]:
users_reliability.count()

5419

# Don't forget to clean up!
## 1) Check column names
## 2) Sort

In [37]:
from pyspark.sql.functions import desc

users_reliability_sorted = users_reliability.select(["userId", col("avg(reliability)").alias("reliability")]).sort(desc("reliability"))
users_reliability_sorted.show()

+------+------------------+
|userId|       reliability|
+------+------------------+
| 81949|0.9999997749296758|
| 50441|0.9999997749296758|
|136455|0.9999997726676818|
|113642|0.9999997726676818|
|119384|0.9999997726676818|
| 23923|0.9999997726676818|
| 68839|0.9999997726676818|
|105357|0.9999997726676818|
|136694|0.9999997726676818|
| 20109| 0.999999772656315|
| 52462|0.9999997712922528|
| 30718|0.9999997703829543|
| 75104|0.9999997703829543|
|112350|0.9999997703829543|
|126600|0.9999997703829543|
| 62725|0.9999997703829543|
| 36433|0.9999997703829543|
| 73204|0.9999997703829543|
| 68905|0.9999997680752648|
|130819|0.9999997680752648|
+------+------------------+
only showing top 20 rows



## 2) Best Movies

<br>
<p style="text-indent: 40px">For the second part of this assignment, we are going to get best 20 movies from those 1000 people's scores. For each movie in the ratings, you should multiply rating with the users score that we previously calculated and get the total of this score for each movie.

In the end we should be able to see the titles of the first 20 movies ranked by this criteria.
</p>




In [38]:
users_reliability_ratings_movies = ratings.join(users_reliability_sorted, on="userId")
users_reliability_ratings_movies.show()

+------+-------+------+-------------------+------------------+
|userId|movieId|rating|          timestamp|       reliability|
+------+-------+------+-------------------+------------------+
|110629|    180|     5|2009-07-02 02:34:33|0.9999983033480123|
|110629|    223|     5|2009-09-27 01:01:58|0.9999983033480123|
|110629|    296|   4.5|2009-06-23 15:41:07|0.9999983033480123|
|110629|    527|   4.5|2009-06-23 03:53:04|0.9999983033480123|
|110629|    593|     5|2009-06-25 16:59:38|0.9999983033480123|
|110629|    858|     5|2009-06-23 15:41:45|0.9999983033480123|
|110629|    904|     5|2009-07-05 05:14:10|0.9999983033480123|
|110629|    909|   4.5|2009-06-29 20:14:10|0.9999983033480123|
|110629|   1089|     5|2009-06-23 15:41:11|0.9999983033480123|
|110629|   1090|     5|2009-06-23 03:52:18|0.9999983033480123|
|110629|   1193|   3.5|2009-06-25 12:13:24|0.9999983033480123|
|110629|   1208|     4|2009-06-28 18:14:32|0.9999983033480123|
|110629|   1213|   4.5|2009-06-23 21:36:14|0.9999983033

In [39]:
movieId_reliableRating = users_reliability_ratings_movies.withColumn("reliable rating", col("rating")*col("reliability"))\
                                                         .select(["movieId", "reliable rating"])
movieId_reliableRating.show()

+-------+------------------+
|movieId|   reliable rating|
+-------+------------------+
|    180| 4.999991516740062|
|    223| 4.999991516740062|
|    296| 4.499992365066055|
|    527| 4.499992365066055|
|    593| 4.999991516740062|
|    858| 4.999991516740062|
|    904| 4.999991516740062|
|    909| 4.499992365066055|
|   1089| 4.999991516740062|
|   1090| 4.999991516740062|
|   1193| 3.499994061718043|
|   1208|3.9999932133920493|
|   1213| 4.499992365066055|
|   1221| 4.499992365066055|
|   1222| 4.999991516740062|
|   1259| 4.499992365066055|
|   1270| 4.499992365066055|
|   1291| 4.999991516740062|
|   1476| 4.499992365066055|
|   1639| 4.499992365066055|
+-------+------------------+
only showing top 20 rows



# Let's get the top 20 movies

In [41]:
movieTitle_reliableRating = movieId_reliableRating.join(movies, on="movieId").select(["title","reliable rating"])
movieTitle_reliableRating.show()

+--------------------+------------------+
|               title|   reliable rating|
+--------------------+------------------+
|     Mallrats (1995)| 4.999991516740062|
|       Clerks (1994)| 4.999991516740062|
| Pulp Fiction (1994)| 4.499992365066055|
|Schindler's List ...| 4.499992365066055|
|Silence of the La...| 4.999991516740062|
|Godfather, The (1...| 4.999991516740062|
|  Rear Window (1954)| 4.999991516740062|
|Apartment, The (1...| 4.499992365066055|
|Reservoir Dogs (1...| 4.999991516740062|
|      Platoon (1986)| 4.999991516740062|
|One Flew Over the...| 3.499994061718043|
|Apocalypse Now (1...|3.9999932133920493|
|   Goodfellas (1990)| 4.499992365066055|
|Godfather: Part I...| 4.499992365066055|
|Full Metal Jacket...| 4.999991516740062|
|  Stand by Me (1986)| 4.499992365066055|
|Back to the Futur...| 4.499992365066055|
|Indiana Jones and...| 4.999991516740062|
|Private Parts (1997)| 4.499992365066055|
|  Chasing Amy (1997)| 4.499992365066055|
+--------------------+------------

In [42]:
top_movies = movieTitle_reliableRating.sort(desc("reliable rating"))
top_movies.show()

+--------------------+-----------------+
|               title|  reliable rating|
+--------------------+-----------------+
|Pan's Labyrinth (...|4.999998874648379|
|Amelie (Fabuleux ...|4.999998874648379|
|Love Actually (2003)|4.999998874648379|
|    Happiness (1998)|4.999998874648379|
|Ricky Gervais Liv...|4.999998874648379|
|Usual Suspects, T...|4.999998874648379|
|     Magnolia (1999)|4.999998874648379|
|Fifth Element, Th...|4.999998874648379|
|       Snatch (2000)|4.999998874648379|
|Barbarian Invasio...|4.999998874648379|
|Amores Perros (Lo...|4.999998874648379|
|Kill Bill: Vol. 2...|4.999998874648379|
|Ricky Gervais Liv...|4.999998874648379|
|Kikujiro (Kikujir...|4.999998874648379|
|Spirited Away (Se...|4.999998874648379|
|Me and You and Ev...|4.999998874648379|
|Usual Suspects, T...|4.999998874648379|
|Lost in Translati...|4.999998874648379|
| Pulp Fiction (1994)|4.999998874648379|
|Gone with the Win...|4.999998863338409|
+--------------------+-----------------+
only showing top

# Best movie!
<img src='https://www.edgehill.ac.uk/events/files/2017/12/Pans-Labyrinth.jpg' />